In [1]:
#!/bin/bash
!curl -L -o /content/archive.zip https://www.kaggle.com/api/v1/datasets/download/kmader/electron-microscopy-3d-segmentation

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  214M  100  214M    0     0   136M      0  0:00:01  0:00:01 --:--:--  243M


In [2]:
!unzip -o /content/archive.zip

Archive:  /content/archive.zip
  inflating: testing.tif             
  inflating: testing_groundtruth.tif  
  inflating: training.tif            
  inflating: training_groundtruth.tif  


In [3]:
import tifffile
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import Accuracy

# Load data function
def load_data():
    # Load the TIFF images
    train_images = tifffile.imread('/content/training.tif')
    train_labels = tifffile.imread('/content/training_groundtruth.tif')
    test_images = tifffile.imread('/content/testing.tif')
    test_labels = tifffile.imread('/content/testing_groundtruth.tif')

    # Normalize the images to [0, 1]
    train_images = train_images.astype(np.float32) / np.max(train_images)
    test_images = test_images.astype(np.float32) / np.max(test_images)

    # Ensure labels are binary (0 or 1)
    train_labels = (train_labels > 0).astype(np.float32)
    test_labels = (test_labels > 0).astype(np.float32)

    # Add channel dimension for compatibility with Keras (depth, height, width, channels)
    train_images = np.expand_dims(train_images, axis=-1)
    train_labels = np.expand_dims(train_labels, axis=-1)
    test_images = np.expand_dims(test_images, axis=-1)
    test_labels = np.expand_dims(test_labels, axis=-1)

    return train_images, train_labels, test_images, test_labels

# Load the data
train_images, train_labels, test_images, test_labels = load_data()

# Resize images to (256, 256) and add a channel dimension for grayscale (1 channel)
train_images = tf.image.resize(train_images, (256, 256))
train_labels = tf.image.resize(train_labels, (256, 256))
test_images = tf.image.resize(test_images, (256, 256))
test_labels = tf.image.resize(test_labels, (256, 256))

train_images = np.expand_dims(train_images, axis=-1)  # Adds the channel dimension
test_images = np.expand_dims(test_images, axis=-1)  # Adds the channel dimension

# Ensure labels are clipped between 0 and 1
train_labels = np.clip(train_labels, 0, 1)
test_labels = np.clip(test_labels, 0, 1)

# Check if the shapes are now correct
print(train_images.shape)  # Should be (num_samples, 256, 256, 1)
print(test_images.shape)   # Should be (num_samples, 256, 256, 1)
print(train_labels.shape)  # Should be (num_samples, 256, 256, 1)
print(test_labels.shape)   # Should be (num_samples, 256, 256, 1)

# Define the 2D U-Net model architecture
def unet_2d(input_shape):
    inputs = Input(input_shape)

    # Encoding path
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Decoding path
    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = concatenate([up6, conv4], axis=-1)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = concatenate([up7, conv3], axis=-1)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = concatenate([up8, conv2], axis=-1)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = concatenate([up9, conv1], axis=-1)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Output layer
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    # Define the model
    model = Model(inputs=[inputs], outputs=[outputs])
    return model

# Define input shape and initialize the model
input_shape = (256, 256, 1)  # (height, width, channels)
model = unet_2d(input_shape)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryCrossentropy(), metrics=[Accuracy()])

# Train the model
history = model.fit(
    train_images, train_labels,
    validation_data=(test_images, test_labels),
    epochs=10,
    batch_size=8
)

# Evaluate on the test set
model.evaluate(test_images, test_labels)

# Predict on test images
predicted_labels = model.predict(test_images)

import matplotlib.pyplot as plt
import imageio

# Save each slice as a frame in a GIF
frames = []
for i in range(predicted_labels.shape[0]):  # Loop through the images in the batch
    fig, ax = plt.subplots()
    ax.imshow(predicted_labels[i, :, :, 0], cmap='gray')  # Adjust the indices based on shape
    ax.axis('off')

    # Save frame to a file-like object for creating the GIF
    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames.append(frame)
    plt.close(fig)

# Create a GIF
gif_path = 'segmentation_results.gif'
imageio.mimsave(gif_path, frames, fps=5)

print(f"GIF saved at: {gif_path}")

(165, 256, 256, 1, 1)
(165, 256, 256, 1, 1)
(165, 256, 256, 1)
(165, 256, 256, 1)
Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.0000e+00 - loss: 0.5266 - val_accuracy: 0.0000e+00 - val_loss: 0.2128
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.0000e+00 - loss: 0.2193 - val_accuracy: 0.0000e+00 - val_loss: 0.2074
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.1931 - val_accuracy: 0.0000e+00 - val_loss: 0.2088
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.1741 - val_accuracy: 0.0000e+00 - val_loss: 0.2183
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.0000e+00 - loss: 0.1323 - val_accuracy: 0.0000e+00 - val_loss: 0.1588
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.0000e+00 - loss: 0.0964 - val_accuracy: 0.0000e+00 - val_loss: 0.1699
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.0000e+00 - loss: 0.0814 - val_accuracy: 0.00

<ipython-input-3-81191cc829d1>:142: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')


GIF saved at: segmentation_results.gif


In [4]:
model.save("Segmanation.keras")

In [5]:
from tensorflow.keras.models import load_model
# The dataset is primarily used for image segmentation, where the goal is to identify and delineate different structures (e.g., mitochondria or synapses) within the brain tissue.
# The challenge is to train machine learning models (like neural networks) to perform this segmentation accurately.
#

model = load_model("Segmanation.keras")


# Load data function
def load_data():
  # Load the TIFF images
  custom_img = tifffile.imread('/content/testing.tif')
  custom_labels = tifffile.imread('/content/testing_groundtruth.tif')

  # Normalize the images to [0, 1]
  custom_img = custom_img.astype(np.float32) / np.max(train_images)
  # Ensure labels are binary (0 or 1)
  custom_labels = (custom_labels > 0).astype(np.float32)

  # Add channel dimension for compatibility with Keras (depth, height, width, channels)
  custom_img = np.expand_dims(custom_img, axis=-1)
  custom_labels = np.expand_dims(custom_labels, axis=-1)

  return custom_img, custom_labels

# Load the data
current_img, current_labels = load_data()

# Resize images to (256, 256) and add a channel dimension for grayscale (1 channel)
current_img = tf.image.resize(current_img, (256, 256))
current_labels = tf.image.resize(current_labels, (256, 256))

current_img = np.expand_dims(current_img, axis=-1)  # Adds the channel dimension

# Ensure labels are clipped between 0 and 1
current_labels = np.clip(current_labels, 0, 1)

# Check if the shapes are now correct
print(current_img.shape)  # Should be (num_samples, 256, 256, 1)
print(current_labels.shape)   # Should be (num_samples, 256, 256, 1)

# Predict on test images
predicted_labels = model.predict(current_img)

import matplotlib.pyplot as plt
import imageio

# Save each slice as a frame in a GIF
frames = []
for i in range(predicted_labels.shape[0]):  # Loop through the images in the batch
    fig, ax = plt.subplots()
    ax.imshow(predicted_labels[i, :, :, 0], cmap='gray')  # Adjust the indices based on shape
    ax.axis('off')

    # Save frame to a file-like object for creating the GIF
    fig.canvas.draw()
    frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    frame = frame.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    frames.append(frame)
    plt.close(fig)

# Create a GIF
gif_path = 'segmentation_resultsfinal2.gif'
imageio.mimsave(gif_path, frames, fps=5)

print(f"GIF saved at: {gif_path}")

(165, 256, 256, 1, 1)
(165, 256, 256, 1)
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 173ms/step


<ipython-input-5-156f08021aaa>:57: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed two minor releases later. Use buffer_rgba instead.
  frame = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')


GIF saved at: segmentation_resultsfinal2.gif
